In [4]:
import numpy as np
import pyaudio
import os
import struct
import wave
from kafka import KafkaProducer
from time import sleep
import json

In [29]:
'''
    Data needed = 4 seconds
    Rate = 44100 sample/sec
    Chunk Size = 1024
    Run loop for (44100/1024) * 4 times
    Pad with zeros = 4*(44100/2) - len(recording)
    {divide by 2 because of single channel} {88200 samples finally}
'''

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 4
PAD_LENGTH = (RATE//2)*RECORD_SECONDS

In [30]:
p = pyaudio.PyAudio()

In [31]:
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

In [32]:
producer = KafkaProducer(bootstrap_servers = ['localhost:9092'])

In [ ]:
while True:
    data = stream.read(CHUNK)
    producer.send('test',value = data)

In [27]:
stream.stop_stream()
stream.close()
p.terminate()

In [99]:


p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []
decodedData = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    decodedData.append(np.frombuffer(data, np.float32))
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [104]:

decodedData = np.hstack(decodedData)
newAudio = np.append(decodedData,np.zeros(PAD_LENGTH-len(decodedData)))

In [105]:
newAudio.shape

(88200,)

In [110]:
ar = np.array([1,2,3,4])
ar = np.append(ar[1:],np.array([5]))
ar

array([2, 3, 4, 5])

In [74]:
import librosa

In [101]:
data,sr = librosa.load(WAVE_OUTPUT_FILENAME)
print(data.shape,sr,len(data)/sr)

(88064,) 22050 3.9938321995464854


In [102]:
data

array([0.00079353, 0.00066436, 0.00037531, ..., 0.00325616, 0.00342575,
       0.00405194], dtype=float32)